In [ ]:
%matplotlib inline

import pydata_google_auth
import pandas as pd
import pandas_gbq

import matplotlib.pyplot as plt

projectid = "pangeo-181919"
table = 'pangeo-181919.pangeo_kubernetes_logs.gcp_billing_export_v1_016C8D_761AEE_B0C379'

In [ ]:
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/cloud-platform'],
)
pandas_gbq.context.credentials = credentials

In [ ]:
plt.rcParams['figure.figsize'] = (16,10)
plt.rcParams['font.size'] = 14

In [ ]:
def query_to_df(query):
    return pd.read_gbq(query, project_id=projectid, dialect='standard')

def plot_result(df, target='cost', gb_cols=None):
    if gb_cols is None:
        gb_cols = [col for col in df.columns if col != target]
    # make sure time is first for gb
    if 'time' in gb_cols:
        gb_cols.remove('time')
        gb_cols = ['time'] + gb_cols
    df_gb = df.groupby(gb_cols)[target].sum()
    fig, ax = plt.subplots()
    df_gb.unstack().plot(kind='area', stacked=True, ax=ax)
    ax.grid()
    ax.set_ylabel('USD')
    return ax


In [ ]:
query = """
#standardSQL
SELECT
    service.description as service_description,
    EXTRACT(DATE from usage_start_time) as time,
    SUM(cost) as cost
FROM `pangeo-181919.pangeo_kubernetes_logs.gcp_billing_export_v1_016C8D_761AEE_B0C379`
GROUP BY service_description, time
ORDER BY cost DESC
"""

df_services = query_to_df(query)
ax = plot_result(df_services)
ax.set_title('Pangeo GCP Service Cost')

In [ ]:
query = """
#standardSQL
SELECT * from (
  SELECT
    labels.value as cluster,
    sku.description as sku_description,
    EXTRACT(DATE from usage_start_time) as time,
    SUM(cost) as cost
  FROM `pangeo-181919.pangeo_kubernetes_logs.gcp_billing_export_v1_016C8D_761AEE_B0C379`
  LEFT JOIN UNNEST(labels) as labels
    ON labels.key = "cluster"
  GROUP BY cluster, sku_description, time)
WHERE
 (cluster IS NOT NULL) and
 (cost > 1)
ORDER BY cluster, cost DESC
"""


df_clusters = query_to_df(query)
ax = plot_result(df_clusters, gb_cols=['time', 'cluster'])
ax.set_title('Pangeo GCP Cluster Cost')

In [ ]:
top_clusters = df_clusters.groupby('cluster')['cost'].sum().nlargest(3)
top_clusters

In [ ]:
for cluster in top_clusters.index:
    df_cluster = df_clusters[df_clusters.cluster==cluster]
    ax = plot_result(df_cluster, gb_cols=['time', 'sku_description'])
    ax.set_title('Pangeo GCP Cluster Cost')
    ax.set_title(f'{cluster} Cluster Daily Compute Cost')    